In [ ]:
from spade import agent, quit_spade

class BaseAgent(agent.Agent):
    async def setup(self):
        print("Hello World! I'm agent {}".format(str(self.jid)))

BaseAgent1 = BaseAgent("user1@localhost", "user1")
future = BaseAgent1.start()
future.result()

BaseAgent1.stop()
quit_spade()


In [ ]:
import time
import asyncio
from spade.agent import Agent
from spade.behaviour import CyclicBehaviour

class DummyAgent(Agent):
    class MyBehav(CyclicBehaviour):
        async def on_start(self):
            print("Starting behaviour . . .")
            self.counter = 0

        async def run(self):
            print("Counter: {}".format(self.counter))
            self.counter += 1
            await asyncio.sleep(1)

    async def setup(self):
        print("Agent starting . . .")
        b = self.MyBehav()
        self.add_behaviour(b)

if __name__ == "__main__":
    dummy = DummyAgent("user2@localhost", "user2")
    future = dummy.start()
    future.result()

    print("Wait until user interrupts with ctrl+C")
    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        print("Stopping...")
    dummy.stop()


In [1]:
import datetime
import getpass
from re import A
import time
import asyncio
import spade
import xml.etree.ElementTree as ET
import itertools as IT
import requests

from lxml import etree
from spade import quit_spade
from spade.agent import Agent
from spade.behaviour import CyclicBehaviour, PeriodicBehaviour, FSMBehaviour, State, OneShotBehaviour
from spade.message import Message
from spade.template import Template

files = 'Test.XML'


def GenerateDroneDataXML(filename, baseAgent, droneAgent, taskID):
    root = ET.Element("Response")

    droneType = ET.SubElement(root, "DroneType")
    droneType.text = "Drone Agent"

    ontology = ET.SubElement(root, "Ontology")
    ontology.text = "Drone Data"

    droneName = ET.SubElement(root, "Name")
    if str(droneAgent) == "user2@localhost"  : 
        droneName.text = "Drone 1"
    elif str(droneAgent) == "user3@localhost":
        droneName.text = "Drone 2"

    droneID = ET.SubElement(root, "DroneID")
    droneID.text = droneAgent

    locationLatitiude = ET.SubElement(root, "locationLatitiude")
    locationLatitiude.text = "100000"

    locationLongtitude= ET.SubElement(root,"locationLongtitude")
    locationLongtitude.text = "200000"
    
    elevationLevel = ET.SubElement(root,"elevationLevel")
    elevationLevel.text = "5000"

    batteryStatus = ET.SubElement(root,"batteryStatus")
    batteryStatus.text = "100"


    verticalSpeed = ET.SubElement(root,"verticalSpeed")
    verticalSpeed.text = "100"

    horizontalSpeed = ET.SubElement(root,"horizontalSpeed")
    horizontalSpeed.text = "100"

    replyTask = ET.SubElement(root, "ReplyTask")
    replyTask.text = taskID #replace with Task ID

    print(ET.tostring(root))
    tree = ET.ElementTree(root)
    ET.indent(tree)
    tree.write(str(filename), encoding='unicode')
    #tree = ET.tostring(tree).decode()
    #return tree
    #headers={'Content-Type' : 'Test/XML'}

    #requests.post('http://127.0.0.1', data = tree, headers=headers)
    #with open (filename, "wb") as files:
     #   tree.write(files)

def GenerateDroneReturnXML(filename, taskID, baseAgent, droneAgent):
    root = ET.Element("Response")

    droneType = ET.SubElement(root, "DroneType")
    droneType.text = "Drone Agent"

    ontology = ET.SubElement(root, "Ontology")
    ontology.text = "Return Home"

    droneName = ET.SubElement(root, "Name")
    if str(droneAgent) == "user2@localhost"  : 
        droneName.text = "Drone 1"
    elif str(droneAgent) == "user3@localhost":
        droneName.text = "Drone 2"

    droneID = ET.SubElement(root, "DroneID")
    droneID.text = droneAgent

    replyTask = ET.SubElement(root, "ReplyTask")
    replyTask.text = taskID #replace with Task ID

    print(ET.tostring(root))
    tree = ET.ElementTree(root)
    ET.indent(tree)
    tree.write(filename, encoding='unicode')


'''
query drone data
order return home
send drone data
tell return home
send coordinates to work

'''
def GenerateTaskQueryXML(filename, iDCounter, baseAgent, droneAgent):
    root = ET.Element("Task")

    taskID = ET.SubElement(root, "TaskID")
    taskID.text = "T" + str(iDCounter) #ID dynamic

    senderAgent = ET.SubElement(root, "senderAgent")
    senderAgent.text = baseAgent #replace with jID

    receiverAgent = ET.SubElement(root, "receiverAgent")
    receiverAgent.text = droneAgent #replace with jID

    droneName = ET.SubElement(root, "Name")
    if str(droneAgent) == "user2@localhost"  : 
        droneName.text = "Drone 1"
    elif str(droneAgent) == "user3@localhost":
        droneName.text = "Drone 2"

    ontology = ET.SubElement(root, "Ontology")
    ontology.text = "Query Data" 

    taskName = ET.SubElement(root, "TaskName")
    taskName.text = "Request Data from Drone Agent 1 or 2"

    generatedTime =  ET.SubElement(root, "GeneratedTime")
    generatedTime.text = "time here"
    
    tree = ET.ElementTree(root)
    ET.indent(tree)
    tree.write('QueryDrone.XML', encoding='unicode')


def GenerateTaskReturnXML(filename, iDCounter, baseAgent, droneAgent):
    root = ET.Element("Task")

    taskID = ET.SubElement(root, "TaskID")
    taskID.text = "T" + str(iDCounter)

    senderAgent = ET.SubElement(root, "senderAgent")
    senderAgent.text = baseAgent #replace with jID

    receiverAgent = ET.SubElement(root, "receiverAgent")
    receiverAgent.text = droneAgent #replace with jID

    droneName = ET.SubElement(root, "Name")
    if str(droneAgent) == "user2@localhost"  : 
        droneName.text = "Drone 1"
    elif str(droneAgent) == "user3@localhost":
        droneName.text = "Drone 2"

    ontology = ET.SubElement(root, "Ontology")
    ontology.text = "Return Home" 

    taskName = ET.SubElement(root, "TaskName")
    taskName.text = "Request Drone Agent 1 or 2 to return to base"

    generatedTime =  ET.SubElement(root, "GeneratedTime")
    generatedTime.text = "time here"

    tree = ET.ElementTree(root)
    ET.indent(tree)
    tree.write(filename, encoding='unicode')

#need to put two coordinates (latitude, longitude) here
def GenerateTaskGoCoordinatesXML(filename, iDCounter, latitude, longitude, baseAgent, droneAgent):
    root = ET.Element("Task")

    taskID = ET.SubElement(root, "TaskID")
    taskID.text = "T" + str(iDCounter)

    senderAgent = ET.SubElement(root, "senderAgent")
    senderAgent.text = baseAgent #replace with jID

    receiverAgent = ET.SubElement(root, "receiverAgent")
    receiverAgent.text = droneAgent #replace with jID

    droneName = ET.SubElement(root, "Name")
    if str(droneAgent) == "user2@localhost"  : 
        droneName.text = "Drone 1"
    elif str(droneAgent) == "user3@localhost":
        droneName.text = "Drone 2"

    ontology = ET.SubElement(root, "Ontology")
    ontology.text = "Coordinate" 

    taskName = ET.SubElement(root, "TaskName")
    taskName.text = "Drone Agent will move to (", latitude, ", ", longitude, ")"

    statedLatitude = ET.SubElement(root, "Latitude")
    statedLatitude.text = latitude

    statedLongitude = ET.SubElement(root, "Longitude")
    statedLongitude.text = longitude

    generatedTime =  ET.SubElement(root, "GeneratedTime")
    generatedTime.text = "time here"

    tree = ET.ElementTree(root)
    ET.indent(tree)
    tree.write(filename, encoding='unicode')





class Task():
    def __init__(self, TaskID, TaskName, TaskType, TaskMessage, GeneratedTime, agentID, droneID):
        self.TaskID = TaskID
        self.TaskName = TaskName
        self.TaskType = TaskType
        self.TaskMessage = TaskMessage
        self.GeneratedTime = GeneratedTime
        self.agentID = agentID
        self.droneID = droneID



class BaseAgent(Agent):


    class SubscribePresenceBehav(OneShotBehaviour):
        def on_available(self, jid, stanza):
            print(
                "[{}] Agent {} is available.".format(self.agent.name, jid.split("@")[0])
            )
        
        def on_subscribed(self,jid):
            print(
                "[{}] Agent {} has accepted the subscription.".format(self.agent.name, jid.split("@")[0])
            )
            print(
                "[{}] Contacts List : {}".format(self.agent.name, self.agent.presence.get_contacts())
            )
        
        def on_subscribe(self, jid):
            print(
                "[{}] Agent {} asked for subscription. Should I approve it?".format(self.agent.name, jid.split("@")[0])
            )
            self.presence.approve(jid)
        
        async def run(self):
            self.presence.on_subscribe = self.on_subscribe
            self.presence.on_subscribed = self.on_subscribed
            self.presence.on_available = self.on_available

            self.presence.set_available()
            self.presence.subscribe(self.agent.DroneAgent1)





    class QueryDroneBehav(PeriodicBehaviour):
        async def run(self):
            self.sequence += 1
            filename = "QueryDrone.XML"
            GenerateTaskQueryXML(filename, self.sequence, "user1@localhost", "user2@localhost")

            print(f"QuerySenderBehaviour running at {datetime.datetime.now().time()}: {self.counter}")

            generatedQuery = ET.parse('QueryDrone.XML')
            rootGeneratedQuery = generatedQuery.getroot()
            droneDataXML = ET.tostring(rootGeneratedQuery).decode()

            msg = Message(to="user2@localhost")  # Instantiate the message
            msg.set_metadata("performative", "request")
            msg.body = droneDataXML  # Set the message content
           
            await self.send(msg)
            print("Base Message sent!")

            if self.counter == 5:
                self.kill()

            asyncio.sleep(1)
            self.counter += 1

        async def on_end(self):
            # stop agent from behaviour
            await self.agent.stop()

        async def on_start(self):
            self.counter = 0
            self.sequence = 0







    class ReceiveDroneDataBehav(CyclicBehaviour):
        async def run(self):
            print(f"QuerySenderBehaviour running at {datetime.datetime.now().time()}")

            receiveDroneMsg = await self.receive(timeout = 10)
            if receiveDroneMsg:
                if str(receiveDroneMsg.sender) == "user2@localhost":
                    #here print the collected data from drones 
                    receivedTree = ET.XML(receiveDroneMsg.body)
                    with open("Drone2Data.XML", "wb") as f:
                        f.write(ET.tostring(receivedTree))
                    treeReceivedData = ET.parse('Drone2Data.XML')
                    rootReceivedData = treeReceivedData.getroot()
                    print(rootReceivedData.tag)
                else:
                    print(receiveDroneMsg.sender)
                
            else:
                print("i did not get any reply")

    async def on_end(self):
            await self.agent.stop()

    #def getCoordinatesInput():

    #async def 

    
    async def setup(self):
        print("Base agent activated")
        b = self.QueryDroneBehav(period = 5)
        self.add_behaviour(b)
        templateA = Template()
        templateA.set_metadata("performative", "inform")
        c = self.ReceiveDroneDataBehav()
        self.add_behaviour(c, templateA)
        d = self.SubscribePresenceBehav()
        self.add_behaviour(d)



class DroneAgent(Agent):

    class SubscribePresenceBehav(OneShotBehaviour):
            def on_available(self, jid, stanza):
                print(
                    "[{}] Agent {} is available.".format(self.agent.name, jid.split("@")[0])
                )
            
            def on_subscribed(self,jid):
                print(
                    "[{}] Agent {} has accepted the subscription.".format(self.agent.name, jid.split("@")[0])
                )
                print(
                    "[{}] Contacts List : {}".format(self.agent.name, self.agent.presence.get_contacts())
                )
            
            def on_subscribe(self, jid):
                print(
                    "[{}] Agent {} asked for subscription. Should I approve it?".format(self.agent.name, jid.split("@")[0])
                )
                self.presence.approve(jid)
                self.presence.subscribe(jid)

            async def run(self):
                self.presence.on_subscribe = self.on_subscribe
                self.presence.on_subscribed = self.on_subscribed
                self.presence.on_available = self.on_available

                self.presence.set_available()
                self.presence.subscribe(self.agent.BaseAgent)




    class ReceiveOrder(CyclicBehaviour): 
        async def run(self):
            self.contRecvOrder = True
            while(self.contRecvOrder):
                recvMsg = await self.receive(timeout = 10)

                receivedTree = ET.XML(recvMsg.body)
                with open("Task.XML", "wb") as f:
                    f.write(ET.tostring(receivedTree))
                treeReceivedData = ET.parse('Task.XML')
                rootReceivedData = treeReceivedData.getroot()
                print(rootReceivedData)
                for elem in rootReceivedData.iter('Ontology'):
                    self.ontology = str(elem.text)
                print(self.ontology)
                if (self.ontology == "Query Data"):
                    GenerateDroneDataXML("DroneData.XML", "user1@localhost", self.agent.name, "T1")
                    tree1 = ET.parse('DroneData.XML')
                    root1 = tree1.getroot()
                    droneDataXML = ET.tostring(root1).decode()


                    msg = Message(to="user1@localhost")  # Instantiate the message
                    msg.set_metadata("performative", "inform")
                    msg.body = droneDataXML

                    await self.send(msg)
            
        async def on_end(self):
            await self.agent.stop()

    # use async def function to call XML  and put into task


    async def setup(self):
        print("Drone Agent initiated")
        recOrder = self.ReceiveOrder()
        template = Template()
        template.set_metadata("performative", "request")
        self.add_behaviour(recOrder, template)
        a = self.SubscribePresenceBehav()
        self.add_behaviour(a)



if __name__ == "__main__":

    
    droneagent = DroneAgent("user2@localhost", "user2")
    baseagent = BaseAgent("user1@localhost", "user1")

    baseagent.DroneAgent1 = "user2@localhost"
    droneagent.BaseAgent = "user1@localhost"

    future = droneagent.start(auto_register = True)
    future.result()

    
    future2 = baseagent.start(auto_register = True)
    future2.result()


    while droneagent.is_alive():
        try:
            await asyncio.sleep(5)
        except KeyboardInterrupt:
            baseagent.stop()
            droneagent.stop()
            break
    print("Agents finished")
    quit_spade()

Drone Agent initiated
Base agent activated
QuerySenderBehaviour running at 17:19:27.074854: 0
Base Message sent!
QuerySenderBehaviour running at 17:19:27.081312
<Element 'Task' at 0x7f2c283b1040>
Query Data
b'<Response><DroneType>Drone Agent</DroneType><Ontology>Drone Data</Ontology><Name /><DroneID>user2</DroneID><locationLatitiude>100000</locationLatitiude><locationLongtitude>200000</locationLongtitude><elevationLevel>5000</elevationLevel><batteryStatus>100</batteryStatus><verticalSpeed>100</verticalSpeed><horizontalSpeed>100</horizontalSpeed><ReplyTask>T1</ReplyTask></Response>'
Response
QuerySenderBehaviour running at 17:19:27.110222


/tmp/ipykernel_2934/2818110425.py:282: RuntimeWarning: coroutine 'sleep' was never awaited
  asyncio.sleep(1)


QuerySenderBehaviour running at 17:19:32.075283: 1
Base Message sent!
<Element 'Task' at 0x7f2c283b1950>
Query Data
b'<Response><DroneType>Drone Agent</DroneType><Ontology>Drone Data</Ontology><Name /><DroneID>user2</DroneID><locationLatitiude>100000</locationLatitiude><locationLongtitude>200000</locationLongtitude><elevationLevel>5000</elevationLevel><batteryStatus>100</batteryStatus><verticalSpeed>100</verticalSpeed><horizontalSpeed>100</horizontalSpeed><ReplyTask>T1</ReplyTask></Response>'
Response
QuerySenderBehaviour running at 17:19:32.077460
QuerySenderBehaviour running at 17:19:37.075549: 2
Base Message sent!
<Element 'Task' at 0x7f2c283c7950>
Query Data
b'<Response><DroneType>Drone Agent</DroneType><Ontology>Drone Data</Ontology><Name /><DroneID>user2</DroneID><locationLatitiude>100000</locationLatitiude><locationLongtitude>200000</locationLongtitude><elevationLevel>5000</elevationLevel><batteryStatus>100</batteryStatus><verticalSpeed>100</verticalSpeed><horizontalSpeed>100</h

In [2]:
import xml.etree.ElementTree as ET
from lxml import etree
import requests

def GenerateDroneDataXML(filename, baseAgent, droneAgent, taskID):
    root = ET.Element("Response")

    droneType = ET.SubElement(root, "DroneType")
    droneType.text = "Drone Agent"

    ontology = ET.SubElement(root, "Ontology")
    ontology.text = "Drone Data"

    droneName = ET.SubElement(root, "Name")
    if str(droneAgent) == "user2@localhost"  : 
        droneName.text = "Drone 1"
    elif str(droneAgent) == "user3@localhost":
        droneName.text = "Drone 2"

    droneID = ET.SubElement(root, "DroneID")
    droneID.text = droneAgent

    locationLatitiude = ET.SubElement(root, "locationLatitiude")
    locationLatitiude.text = "100000"

    locationLongtitude= ET.SubElement(root,"locationLongtitude")
    locationLongtitude.text = "200000"
    
    elevationLevel = ET.SubElement(root,"elevationLevel")
    elevationLevel.text = "5000"

    batteryStatus = ET.SubElement(root,"batteryStatus")
    batteryStatus.text = "100"


    verticalSpeed = ET.SubElement(root,"verticalSpeed")
    verticalSpeed.text = "100"

    horizontalSpeed = ET.SubElement(root,"horizontalSpeed")
    horizontalSpeed.text = "100"

    replyTask = ET.SubElement(root, "ReplyTask")
    replyTask.text = taskID #replace with Task ID

    print(ET.tostring(root))
    tree = ET.ElementTree(root)
    ET.indent(tree)
    tree.write(filename, encoding='unicode')
    #tree = ET.tostring(tree).decode()
    #return tree
    #headers={'Content-Type' : 'Test/XML'}

    #requests.post('http://127.0.0.1', data = tree, headers=headers)
    #with open (filename, "wb") as files:
     #   tree.write(files)

def GenerateDroneReturnXML(filename, taskID, baseAgent, droneAgent):
    root = ET.Element("Response")

    droneType = ET.SubElement(root, "DroneType")
    droneType.text = "Drone Agent"

    ontology = ET.SubElement(root, "Ontology")
    ontology.text = "Return Home"

    droneName = ET.SubElement(root, "Name")
    if str(droneAgent) == "user2@localhost"  : 
        droneName.text = "Drone 1"
    elif str(droneAgent) == "user3@localhost":
        droneName.text = "Drone 2"

    droneID = ET.SubElement(root, "DroneID")
    droneID.text = droneAgent

    replyTask = ET.SubElement(root, "ReplyTask")
    replyTask.text = taskID #replace with Task ID

    print(ET.tostring(root))
    tree = ET.ElementTree(root)
    ET.indent(tree)
    tree.write(filename, encoding='unicode')


'''
query drone data
order return home
send drone data
tell return home
send coordinates to work

'''
def GenerateTaskQueryXML(filename, iDCounter, baseAgent, droneAgent):
    root = ET.Element("Task")

    taskID = ET.SubElement(root, "TaskID")
    taskID.text = "T", iDCounter #ID dynamic

    senderAgent = ET.SubElement(root, "senderAgent")
    senderAgent.text = baseAgent #replace with jID

    receiverAgent = ET.SubElement(root, "receiverAgent")
    receiverAgent.text = droneAgent #replace with jID

    droneName = ET.SubElement(root, "Name")
    if str(droneAgent) == "user2@localhost"  : 
        droneName.text = "Drone 1"
    elif str(droneAgent) == "user3@localhost":
        droneName.text = "Drone 2"

    ontology = ET.SubElement(root, "Ontology")
    ontology.text = "Query Data" 

    taskName = ET.SubElement(root, "TaskName")
    taskName.text = "Request Data from Drone Agent 1 or 2"

    generatedTime =  ET.SubElement(root, "GeneratedTime")
    generatedTime.text = "time here"
    
    tree = ET.ElementTree(root)
    ET.indent(tree)
    tree.write(filename, encoding='unicode')


def GenerateTaskReturnXML(filename, iDCounter, baseAgent, droneAgent):
    root = ET.Element("Task")

    taskID = ET.SubElement(root, "TaskID")
    taskID.text = "T", iDCounter

    senderAgent = ET.SubElement(root, "senderAgent")
    senderAgent.text = baseAgent #replace with jID

    receiverAgent = ET.SubElement(root, "receiverAgent")
    receiverAgent.text = droneAgent #replace with jID

    droneName = ET.SubElement(root, "Name")
    if str(droneAgent) == "user2@localhost"  : 
        droneName.text = "Drone 1"
    elif str(droneAgent) == "user3@localhost":
        droneName.text = "Drone 2"

    ontology = ET.SubElement(root, "Ontology")
    ontology.text = "Return Home" 

    taskName = ET.SubElement(root, "TaskName")
    taskName.text = "Request Drone Agent 1 or 2 to return to base"

    generatedTime =  ET.SubElement(root, "GeneratedTime")
    generatedTime.text = "time here"

    tree = ET.ElementTree(root)
    ET.indent(tree)
    tree.write(filename, encoding='unicode')

#need to put two coordinates (latitude, longitude) here
def GenerateTaskGoCoordinatesXML(filename, iDCounter, latitude, longitude, baseAgent, droneAgent):
    root = ET.Element("Task")

    taskID = ET.SubElement(root, "TaskID")
    taskID.text = "T", iDCounter

    senderAgent = ET.SubElement(root, "senderAgent")
    senderAgent.text = baseAgent #replace with jID

    receiverAgent = ET.SubElement(root, "receiverAgent")
    receiverAgent.text = droneAgent #replace with jID

    droneName = ET.SubElement(root, "Name")
    if str(droneAgent) == "user2@localhost"  : 
        droneName.text = "Drone 1"
    elif str(droneAgent) == "user3@localhost":
        droneName.text = "Drone 2"

    ontology = ET.SubElement(root, "Ontology")
    ontology.text = "Coordinate" 

    taskName = ET.SubElement(root, "TaskName")
    taskName.text = "Drone Agent will move to (", latitude, ", ", longitude, ")"

    statedLatitude = ET.SubElement(root, "Latitude")
    statedLatitude.text = latitude

    statedLongitude = ET.SubElement(root, "Longitude")
    statedLongitude.text = longitude

    generatedTime =  ET.SubElement(root, "GeneratedTime")
    generatedTime.text = "time here"

    tree = ET.ElementTree(root)
    ET.indent(tree)
    tree.write(filename, encoding='unicode')











if __name__ == "__main__":
    GenerateDroneDataXML("Test.XML", "user1@localhost", "user2@localhost", "T1")
    #print(xml_string)
    parser = etree.XMLParser(recover=True)
    tree1 = ET.parse('Test.XML')
    root1 = tree1.getroot()
    r = ET.tostring(root1).decode()
    print(type(r))

    #print(tree1)
    print(root1.tag)
    for elem in root1.iter():
        for elem1 in elem:
            print(elem1.tag, "<" , elem1.text, ">")

    
    #requests.posts('user2@localhost', data = )

b'<Response><DroneType>Drone Agent</DroneType><Ontology>Drone Data</Ontology><Name>Drone 1</Name><DroneID>user2@localhost</DroneID><locationLatitiude>100000</locationLatitiude><locationLongtitude>200000</locationLongtitude><elevationLevel>5000</elevationLevel><batteryStatus>100</batteryStatus><verticalSpeed>100</verticalSpeed><horizontalSpeed>100</horizontalSpeed><ReplyTask>T1</ReplyTask></Response>'
<class 'str'>
Response
DroneType < Drone Agent >
Ontology < Drone Data >
Name < Drone 1 >
DroneID < user2@localhost >
locationLatitiude < 100000 >
locationLongtitude < 200000 >
elevationLevel < 5000 >
batteryStatus < 100 >
verticalSpeed < 100 >
horizontalSpeed < 100 >
ReplyTask < T1 >
